### Package imports

In [ ]:
# Import packages
import re
import os
import copy
from pathlib import Path
import numpy as np
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
import pandas as pd
import random
import datetime
import time
from pdb import set_trace
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import BayesianGaussianMixture
import matplotlib.pyplot as plt
import gc

### Set seed function

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    rs = RandomState(MT19937(SeedSequence(seed)))

### File path

In [ ]:
# File path
raw_data_path = '/home/ec2-user/MLNotebook shared/Datasets/Test set raw data/' # for Johansson and Mertins raw data, 
dataset_folder_path = '/home/ec2-user/MLNotebook shared/Datasets/' # for localization label
output_path = '/home/ec2-user/MLNotebook shared/Datasets/'

### Preparing data for Krug

#### Loading in data

In [ ]:
# Open the labels data
LFP = 'SubCellBarcode.MCF7.txt'
LD = pd.read_csv(filepath_or_buffer=dataset_folder_path+LFP,sep='\t')

# Data set wrangling
LD.index = LD.loc[:,'Protein']
LD = LD.loc[:,LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = LD.loc[:,'Localization'] != 'Unclassified'
LD = LD.loc[NotUnclassInd,:]

In [ ]:
# Open the proteomics data and only keep genes (rows) that are fully quantified
PFP = 'kr_pro_raw.csv' # proteomics file path, normalized by pool, log2 transformed.
PD = pd.read_csv(dataset_folder_path+PFP)

# Data set wrangling
PD.index = PD.loc[:,'Gene']
PD = PD.loc[:,PD.columns!='Gene']
PD.dropna(inplace=True)

# Specific for krug raw ddamsproteomics, these 3 tumors are not in transcriptome data
PD = PD.loc[:,PD.columns!='X11BR057']
PD = PD.loc[:,PD.columns!='X11BR076']
PD = PD.loc[:,PD.columns!='X11BR078']

# Open the mRNA data and only keep genes (rows) that are fully quantified
MFP = 'kr_rna_raw.csv' # mRNA file path, gene centric median normalized, log2 transformed
MD = pd.read_csv(dataset_folder_path+MFP)

# Data set wrangling
MD.index = MD.loc[:,'Gene']
MD = MD.loc[:,MD.columns!='Gene']
MD = MD.drop_duplicates()
MD.dropna(inplace=True)

In [ ]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in PD.index if ((value in MD.index) & (value in LD.index))]
PD = PD.loc[IntersectingGenes,:]
MD = MD.loc[IntersectingGenes,:]
LD = LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('Krug')
print(len(PD.index))
print(len(MD.index))
print(len(LD.index))

#### Normalization

In [ ]:
# Un-log2 transform the proteomics data
PD = 2**PD

# Normalize by gene average
PD = PD.div(PD.mean(axis=1), axis=0)

# Normalize by tumor median
PD = PD.div(PD.median(axis=0), axis=1)

# Log2 transform
PD = np.log2(PD)

# Standardize by dividing gene standard deviation
PD = PD.div(PD.std(axis=1), axis=0)

# Put values of each column in the DataFrame into a list
values = np.sort(PD.values.flatten().tolist())

# Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
PD = (PD) / (percentile_high - percentile_low)

# Un-log2 transform the mRNA data
MD = 2**MD

# Normalize by gene average
MD = MD.div(MD.mean(axis=1), axis=0)

# Normalize by tumor median
MD = MD.div(MD.median(axis=0), axis=1)

# Log2 transform
MD = np.log2(MD)

# Standardize by gene standard deviation
MD = MD.div(MD.std(axis=1), axis=0)

# Put values of each column in the DataFrame into a list
values = np.sort(MD.values.flatten().tolist())

# Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
MD = (MD) / (percentile_high - percentile_low)

### Preparing data for Johansson

#### Loading in data

In [4]:
# Open the localization data
LFP = 'SubCellBarcode.MCF7.txt'
LD = pd.read_csv(filepath_or_buffer=raw_data_path+LFP,sep='\t')

# Data set wrangling
LD.index = LD.loc[:,'Protein']
LD = LD.loc[:,LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = LD.loc[:,'Localization'] != 'Unclassified'
LD = LD.loc[NotUnclassInd,:]

In [ ]:
# Johansson data set
# Open the proteomics data and only keep genes (rows) that are fully quantified
J_PFP = 'jo_protein_log2.csv' # proteomics file path, normalized by pool, log2 transformed.
J_PD = pd.read_csv(raw_data_path+J_PFP)

# Data set wrangling
J_PD.index = J_PD.loc[:,'Unnamed: 0']
J_PD = J_PD.loc[:,J_PD.columns!='Unnamed: 0']
J_PD.dropna(inplace=True)

# Open the mRNA data and only keep genes (rows) that are fully quantified
J_MFP = 'jo_mrna_dropna.csv' # mRNA file path, gene centric median normalized, log2 transformed
J_MD = pd.read_csv(raw_data_path + J_MFP)

# Data set wrangling
J_MD.index = J_MD.loc[:,'Unnamed: 0']
J_MD = J_MD.loc[:,J_MD.columns!='Unnamed: 0']
J_MD = J_MD.drop_duplicates()
J_MD.dropna(inplace=True)

In [ ]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in J_PD.index if ((value in J_MD.index) & (value in LD.index))]
J_PD = J_PD.loc[IntersectingGenes,:]
J_MD = J_MD.loc[IntersectingGenes,:]
LD = LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('Johansson')
print(len(J_PD.index))
print(len(J_MD.index))
print(len(LD.index))

Johansson
7989
7989
7989


#### Normalization

In [9]:
# Un-log2 transform the proteomics data
J_PD = 2**J_PD

# Normalize by gene average
J_PD = J_PD.div(J_PD.mean(axis=1), axis=0)

# Normalize by tumor median
J_PD = J_PD.div(J_PD.median(axis=0), axis=1)

# Log2 transform
J_PD = np.log2(J_PD)

# Standardize by gene standard deviation
J_PD = J_PD.div(J_PD.std(axis=1), axis=0)

# Put values of each column in the DataFrame into a list
values = np.sort(J_PD.values.flatten().tolist())

# Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
J_PD = (J_PD) / (percentile_high - percentile_low)

# Un-log2 transform the mRNA data
J_MD = 2**J_MD

# Normalize by the average of each gene
J_MD = J_MD.div(J_MD.mean(axis=1), axis=0)

# Normalize by the median of each tumor
J_MD = J_MD.div(J_MD.median(axis=0), axis=1)

# Log2 transform
J_MD = np.log2(J_MD)

# Standardize by gene standard deviation
J_MD = J_MD.div(J_MD.std(axis=1), axis=0)

# Put values of each column in the DataFrame into a list
values = np.sort(J_MD.values.flatten().tolist())

# Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
J_MD = (J_MD) / (percentile_high - percentile_low)

In [12]:
# Export the labels for use
LD.to_csv(Path(output_path+'Johansson_Localization.csv'))

### Preparing data for Mertins

In [ ]:
# Open the localization data
LFP = 'SubCellBarcode.MCF7.txt'
LD = pd.read_csv(filepath_or_buffer=raw_data_path+LFP,sep='\t')

# Data set wrangling
LD.index = LD.loc[:,'Protein']
LD = LD.loc[:,LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = LD.loc[:,'Localization'] != 'Unclassified'
LD = LD.loc[NotUnclassInd,:]

In [ ]:
# Mertins data set
# Open the proteomics data and only keep genes (rows) that are fully quantified
M_PFP = 'me_protein_dropna.csv' # proteomics file path, normalized by pool, log2 transformed.
M_PD = pd.read_csv(raw_data_path+M_PFP)

# Data set wrangling
M_PD.index = M_PD.loc[:,'Unnamed: 0']
M_PD = M_PD.loc[:,M_PD.columns!='Unnamed: 0']
M_PD.dropna(inplace=True)

# Open the mRNA data and only keep genes (rows) that are fully quantified
M_MFP = 'me_rna_dropna.csv' # mRNA file path, gene centric median normalized, log2 transformed
M_MD = pd.read_csv(raw_data_path + M_MFP)

# Data set wrangling
M_MD.index = M_MD.loc[:,'Unnamed: 0']
M_MD = M_MD.loc[:,M_MD.columns!='Unnamed: 0']
M_MD = M_MD.drop_duplicates()
M_MD.dropna(inplace=True)

In [ ]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in M_PD.index if ((value in M_MD.index) & (value in LD.index))]
M_PD = M_PD.loc[IntersectingGenes,:]
M_MD = M_MD.loc[IntersectingGenes,:]
LD = LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('Mertin')
print(len(M_PD.index))
print(len(M_MD.index))
print(len(LD.index))

#### Normalization

In [ ]:
# Un-log2 transform the proteomics data
M_PD = 2**M_PD

# Normalize by gene average
M_PD = M_PD.div(M_PD.mean(axis=1), axis=0)

# Normalize by tumor median
M_PD = M_PD.div(M_PD.median(axis=0), axis=1)

# Log2 transform
M_PD = np.log2(M_PD)

# Standardize by gene standard deviation
M_PD = M_PD.div(M_PD.std(axis=1), axis=0)

# Put values of each column in the DataFrame into a list
values = np.sort(M_PD.values.flatten().tolist())

# Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
M_PD = (M_PD) / (percentile_high - percentile_low)

# Un-log2 transform the mRNA data
M_MD = 2**M_MD

# Normalize the mRNA data, first divide by the average of each gene
M_MD = M_MD.div(M_MD.mean(axis=1), axis=0)

# Divide by the median of each column
M_MD = M_MD.div(M_MD.median(axis=0), axis=1)

# Log2 transform
M_MD = np.log2(M_MD)

# Standardize by gene standard deviation
M_MD = M_MD.div(M_MD.std(axis=1), axis=0)

# Put values of each column in the DataFrame into a list
values = np.sort(M_MD.values.flatten().tolist())

# Find the 2.5 and 97.5 percentile
percentile_high = np.percentile(values, 97.5)
percentile_low = np.percentile(values, 2.5)

# Use the percentile for normalization
M_MD = (M_MD) / (percentile_high - percentile_low)

In [ ]:
# Export the labels for use
LD.to_csv(Path(output_path+'Mertins_Localization.csv'))

### Bayesian inference to generate synthetic data

#### Specify dataset

In [13]:
Set = 'Protein + mRNA' # Change it to 'Protein' if only generating proteome synthetic data
Bayesian = True
Canvas_Size = 18

#### Johansson

In [14]:
gc.collect()

51063

In [ ]:
# Johansson Bayesian inference
# Ensure reproducibility
set_seed(43)

if Bayesian:

    if Set == 'Protein' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        PD_bgm = BayesianGaussianMixture(n_components=10, random_state=45) # Assuming the maximum number of clusters in dataset is 5
        J_PD_T = J_PD.T
        PD_bgm.fit(J_PD_T)
    
        # Generate X new synthetic tumors, result is an array
        if Set == 'Protein':
            J_synthetic_PD, _ = PD_bgm.sample(int(Canvas_Size*Canvas_Size-J_PD.columns.size))
        elif Set == 'Protein + mRNA':
            J_synthetic_PD, _ = PD_bgm.sample(int((Canvas_Size*Canvas_Size-J_PD.columns.size*2)/2))
    
        # Transpose back before merging
        J_synthetic_PD = J_synthetic_PD.T

        # Convert the result to a DataFrame
        J_synthetic_PD = pd.DataFrame(J_synthetic_PD.tolist(), index=J_PD.index)

        # Merge the synthetic data with the original data
        J_PD = pd.concat([J_PD, J_synthetic_PD], axis=1)

        # Sanity check for the number of tumors in each dataframe
        
        if Set == 'Protein + mRNA':
            assert len(J_PD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(J_PD.columns) == Canvas_Size*Canvas_Size

        print(f'{Set} - Bayesian Johansson testing PD: {len(J_PD.columns)}')
      
    if Set == 'mRNA' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        MD_bgm = BayesianGaussianMixture(n_components=10, random_state=46) # Assuming the maximum number of clusters in dataset is 5
        J_MD_T = J_MD.T
        MD_bgm.fit(J_MD_T)

        # Generate X new synthetic tumors, result is an array
        if Set == 'mRNA':
            J_synthetic_MD, _ = MD_bgm.sample(int(Canvas_Size*Canvas_Size-J_MD.columns.size))
        if Set == 'Protein + mRNA':
            J_synthetic_MD, _ = MD_bgm.sample(int((Canvas_Size*Canvas_Size-J_MD.columns.size*2)/2))

        # Transpose back before merging
        J_synthetic_MD = J_synthetic_MD.T

        # Convert the result to a DataFrame
        J_synthetic_MD = pd.DataFrame(J_synthetic_MD.tolist(), index=J_MD.index)

        # Merge the synthetic data with the original data
        J_MD = pd.concat([J_MD, J_synthetic_MD], axis=1)
         
        # Sanity check for the number of tumors in each dataframe
        if Set == 'Protein + mRNA':
            assert len(J_MD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(J_MD.columns) == Canvas_Size*Canvas_Size
        print(f'{Set} - Bayesian Johansson testing MD: {len(J_MD.columns)}')

else:
    print('No synthetic data generated')

In [ ]:
# export the synthetic data
if Set == 'Protein + mRNA':
    J_PD.to_csv(Path(output_path + 'J_prot+mRNA_PD_synthetic_new.csv'))
    J_MD.to_csv(Path(output_path + 'J_prot+mRNA_MD_synthetic_new.csv'))
elif Set == 'Protein':
    J_PD.to_csv(Path(output_path + 'J_prot_PD_synthetic_new.csv'))
elif Set == 'mRNA':
    J_MD.to_csv(Path(output_path + 'J_mRNA_MD_synthetic_new.csv'))

#### Mertins

In [ ]:
gc.collect()

In [ ]:
# Mertins Bayesian inference
set_seed(43)

if Bayesian:

    if Set == 'Protein' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        PD_bgm = BayesianGaussianMixture(n_components=15, random_state=47) # Assuming the maximum number of clusters in dataset is 5
        M_PD_T = M_PD.T
        PD_bgm.fit(M_PD_T)
    
        # Generate X new synthetic tumors, result is an array
        if Set == 'Protein':
            M_synthetic_PD, _ = PD_bgm.sample(int(Canvas_Size*Canvas_Size-M_PD.columns.size))
        elif Set == 'Protein + mRNA':
            M_synthetic_PD, _ = PD_bgm.sample(int((Canvas_Size*Canvas_Size-M_PD.columns.size*2)/2))
    
        # Transpose back before merging
        M_synthetic_PD = M_synthetic_PD.T

        # Convert the result to a DataFrame
        M_synthetic_PD = pd.DataFrame(M_synthetic_PD.tolist(), index=M_PD.index)

        # Merge the synthetic data with the original data
        M_PD = pd.concat([M_PD, M_synthetic_PD], axis=1)

        # Sanity check for the number of tumors in each dataframe
        
        if Set == 'Protein + mRNA':
            assert len(M_PD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(M_PD.columns) == Canvas_Size*Canvas_Size

        print(f'{Set} - Bayesian Mertin testing PD: {len(M_PD.columns)}')
      
    if Set == 'mRNA' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        MD_bgm = BayesianGaussianMixture(n_components=15, random_state=48) # Assuming the maximum number of clusters in dataset is 5
        M_MD_T = M_MD.T
        MD_bgm.fit(M_MD_T)

        # Generate X new synthetic tumors, result is an array
        if Set == 'mRNA':
            M_synthetic_MD, _ = MD_bgm.sample(int(Canvas_Size*Canvas_Size-M_MD.columns.size))
        if Set == 'Protein + mRNA':
            M_synthetic_MD, _ = MD_bgm.sample(int((Canvas_Size*Canvas_Size-M_MD.columns.size*2)/2))

        # Transpose back before merging
        M_synthetic_MD = M_synthetic_MD.T

        # Convert the result to a DataFrame
        M_synthetic_MD = pd.DataFrame(M_synthetic_MD.tolist(), index=M_MD.index)

        # Merge the synthetic data with the original data
        M_MD = pd.concat([M_MD, M_synthetic_MD], axis=1)
         
        # Sanity check for the number of tumors in each dataframe
        if Set == 'Protein + mRNA':
            assert len(M_MD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(M_MD.columns) == Canvas_Size*Canvas_Size
        print(f'{Set} - Bayesian Mertin testing MD: {len(M_MD.columns)}')

else:
    print('No synthetic data generated')

In [ ]:
# Export
if Set == 'Protein + mRNA':
    M_PD.to_csv(Path(output_path + 'M_prot+mRNA_PD_synthetic_new.csv'))
    M_MD.to_csv(Path(output_path + 'M_prot+mRNA_MD_synthetic_new.csv'))
elif Set == 'Protein':
    M_PD.to_csv(Path(output_path + 'M_prot_PD_synthetic_new.csv'))
elif Set == 'mRNA':
    M_MD.to_csv(Path(output_path + 'M_mRNA_MD_synthetic_new.csv'))

#### Krug

In [ ]:
# Set seed to ensure reproducibility
set_seed(43)

if Bayesian:

    if Set == 'Protein' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        PD_bgm = BayesianGaussianMixture(n_components=15, random_state=42) # Assuming the maximum number of clusters in dataset is 5
        PD_T = PD.T
        PD_bgm.fit(PD_T)
    
        # Generate X new synthetic tumors, result is an array
        if Set == 'Protein':
            synthetic_PD, _ = PD_bgm.sample(int(Canvas_Size*Canvas_Size-PD.columns.size))
        elif Set == 'Protein + mRNA':
            synthetic_PD, _ = PD_bgm.sample(int((Canvas_Size*Canvas_Size-PD.columns.size*2)/2))
    
        # Transpose back before merging
        synthetic_PD = synthetic_PD.T

        # Convert the result to a DataFrame
        synthetic_PD = pd.DataFrame(synthetic_PD.tolist(), index=PD.index)

        # Merge the synthetic data with the original data
        PD = pd.concat([PD, synthetic_PD], axis=1)

        # Sanity check for the number of tumors in each dataframe
        
        if Set == 'Protein + mRNA':
            assert len(PD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(PD.columns) == Canvas_Size*Canvas_Size

        print(f'{Set} - Bayesian PD: {len(PD.columns)}')
      
    if Set == 'mRNA' or Set == 'Protein + mRNA':
        # Fit the dataset to Bayesian Gaussian Mixture Model
        MD_bgm = BayesianGaussianMixture(n_components=15, random_state=43) # Assuming the maximum number of clusters in dataset is 5
        MD_T = MD.T
        MD_bgm.fit(MD_T)

        # Generate X new synthetic tumors, result is an array
        if Set == 'mRNA':
            synthetic_MD, _ = MD_bgm.sample(int(Canvas_Size*Canvas_Size-MD.columns.size))
        elif Set == 'Protein + mRNA':
            synthetic_MD, _ = MD_bgm.sample(int((Canvas_Size*Canvas_Size-MD.columns.size*2)/2))

        # Transpose back before merging
        synthetic_MD = synthetic_MD.T

        # Convert the result to a DataFrame
        synthetic_MD = pd.DataFrame(synthetic_MD.tolist(), index=MD.index)

        # Merge the synthetic data with the original data
        MD = pd.concat([MD, synthetic_MD], axis=1)
         
        # Sanity check for the number of tumors in each dataframe
        if Set == 'Protein + mRNA':
            assert len(MD.columns) == Canvas_Size*Canvas_Size // 2
        else:
            assert len(MD.columns) == Canvas_Size*Canvas_Size
        print(f'{Set} - Bayesian MD: {len(MD.columns)}')

In [ ]:
# Export
if Set == 'Protein + mRNA':
    PD.to_csv(Path(output_path + 'K_prot+mRNA_PD_synthetic_after.csv'))
    MD.to_csv(Path(output_path + 'K_prot+mRNA_MD_synthetic_after.csv'))
elif Set == 'Protein':
    PD.to_csv(Path(output_path + 'K_prot_PD_synthetic_after.csv'))
elif Set == 'mRNA':
    MD.to_csv(Path(output_path + 'K_mRNA_MD_synthetic_after.csv'))